In [5]:
#引入

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


#何时使用Sequential构建模型
#当每个层正好有一个输入张量和一个输出张量是，可以使用Sequential适用于简单的堆栈堆叠

#如下所示，展示如何使用Sequential构建模型

#定义一个具有三个层的Sequential模型
model = keras.Sequential([
  layers.Dense(2, activation='relu', name='layer1'),
  layers.Dense(3, activation='relu', name='layer2'),
  layers.Dense(4, name='layer3')
])

#在测试输入上调用模型
x = tf.ones((3,3))
y = model(x)

#等效于如下的构建和调用

# 创建三个层
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# 在测试输入上调用模型
x = tf.ones((3, 3))
y = layer3(layer2(layer1(x)))

#Sequential不适合在一下场景构建模型：
#你的模型有多个输入或者输出
#模型中任何一个层具有多个输入或者输出
#你想要共享层的权重
#构建非线性堆叠（拓扑）的方式构建（如，残差连接，多分支模型）

# 创建一个Sequential模型
#你可以将一个层的列表作为Sequential的构造函数来构建模型
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)
#可通过layers属性访问其图层：
model.layers

#您还可以通过add（）方法增量创建一个Sequential模型：
model = keras.Sequential()
model.add(layers.Dense(2, activation="relu"))
model.add(layers.Dense(3, activation="relu"))
model.add(layers.Dense(4))

#特别强调一下，你还可以相应使用pop()方法删除图层：
#Sequential模型的行为非常类似于图层堆栈。

model.pop()
print(len(model.layers))  # 2

#还要注意，就像Keras中的任何层或模型一样，Sequential构造函数也接受name参数。 这对于用语义上有意义的名称注释TensorBoard图很有用。
model = keras.Sequential(name="my_sequential")
model.add(layers.Dense(2, activation="relu", name="layer1"))
model.add(layers.Dense(3, activation="relu", name="layer2"))
model.add(layers.Dense(4, name="layer3"))

#预先指定输入形状
#通常，Keras中的所有图层都需要知道其输入的形状，以便能够创建其权重。 因此，当您创建这样的图层时，最初没有权重：

layer = layers.Dense(3)
layer.weights  # 为空  []

#由于权重的形状取决于输入的形状，因此会在首次调用输入时创建其权重：
# Call layer on a test input
x = tf.ones((1, 4))
y = layer(x)
layer.weights  # Now it has weights, of shape (4, 3) and (3,)

#当然，这也适用于顺序模型。 当实例化没有输入形状的顺序模型时，它不是“构建”的：它没有权重（调用model.weights会导致错误，仅说明这一点）。 权重是在模型首次看到一些输入数据时创建的：

model = keras.Sequential(
    [
        layers.Dense(2, activation="relu"),
        layers.Dense(3, activation="relu"),
        layers.Dense(4),
    ]
)  # No weights at this stage!

# At this point, you can't do this:
# model.weights

# You also can't do this:
# model.summary()

# Call the model on a test input
x = tf.ones((1, 4))
y = model(x)

print("Number of weights after calling the model:", len(model.weights))  # 6

#Number of weights after calling the model: 6

#Once a model is "built", you can call its summary() method to display its contents:
# model.summary()


model = keras.Sequential()
model.add(keras.Input(shape=(4,)))
model.add(layers.Dense(2, activation="relu"))

# model.summary()
model.layers

model = keras.Sequential()
model.add(layers.Dense(2, activation='relu', input_shape=(4,)))
# model.summary()

model = keras.Sequential()
model.add(keras.Input(shape=(250,250,3)))
model.add(layers.Conv2D(32, 5, strides=2, activation='relu'))
model.add(layers.Conv2D(32, 3, activation='relu'))
model.add(layers.MaxPooling2D(3))

model.summary()

model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.Conv2D(32, 3, activation="relu"))
model.add(layers.MaxPooling2D(2))

# And now?
model.summary()

# Now that we have 4x4 feature maps, time to apply global max pooling.
model.add(layers.GlobalMaxPooling2D())

# Finally, we add a classification layer.
model.add(layers.Dense(10))

initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)


initial_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        layers.Conv2D(32, 5, strides=2, activation="relu"),
        layers.Conv2D(32, 3, activation="relu", name="my_intermediate_layer"),
        layers.Conv2D(32, 3, activation="relu"),
    ]
)
feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=initial_model.get_layer(name="my_intermediate_layer").output,
)
# Call feature extractor on test input.
x = tf.ones((1, 250, 250, 3))
features = feature_extractor(x)


2
Number of weights after calling the model: 6
Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 123, 123, 32)      2432      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 121, 121, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 40, 40, 32)        0         
Total params: 11,680
Trainable params: 11,680
Non-trainable params: 0
_________________________________________________________________
